In [1]:
import os
from pyciemss.PetriNetODE.interfaces import (
    load_and_sample_petri_model,
    load_and_calibrate_and_sample_petri_model,
    load_petri_model,
    setup_petri_model,
    sample
)
import numpy as np
from typing import Iterable
from pyciemss.utils.interface_utils import (
    assign_interventions_to_timepoints,
    interventions_and_sampled_params_to_interval,
    convert_to_output_format
)
from torch import tensor

In [2]:
cd ../..

/Users/zuck016/Projects/Proposals/ASKEM/build/pyciemss


In [3]:
DEMO_PATH = "notebook/integration_demo/"
ASKENET_PATH = "https://raw.githubusercontent.com/DARPA-ASKEM/Model-Representations/main/petrinet/examples/sir_typed.json"

## load_and_sample_petri_model

In [4]:
num_samples = 3
timepoints = [0.5, 1.0, 2.0, 3.0, 4.0]

# Run sampling w/o an intervention
samples = load_and_sample_petri_model(ASKENET_PATH, num_samples, timepoints=timepoints)

# Save results
samples.to_csv(os.path.join(DEMO_PATH, "results_petri/sample_results.csv"), index=False)

In [7]:
intervened_samples = load_and_sample_petri_model(
    ASKENET_PATH, num_samples, timepoints=timepoints, interventions=interventions
)


# Save results
intervened_samples.to_csv(
    os.path.join(DEMO_PATH, "results_petri/sample_results_w_interventions.csv"),
    index=False,
)

TypeError: cannot unpack non-iterable StaticParameterInterventionEvent object

## Run sir model step by step

In [4]:
# Run sampling w/o an intervention
# intervention is an iterable (e.g. list) of tuples of the form (timepoint: float, parameter: str, value: float)
interventions = [(1.1, "beta", 1.0), (2.1, "gamma", 0.1), (1.3, "beta", 2.0), (1.4, "gamma", 0.3)]
timepoints = [0.5, 1.0, 2.0, 3.0, 4.0]
num_samples = 3

In [5]:
from pyciemss.PetriNetODE.interfaces import load_petri_model, intervene
from pyciemss.PetriNetODE.base import get_name
raw_model = load_petri_model(ASKENET_PATH)
initial_model = setup_petri_model(raw_model, start_time=-1e-10, start_state = {
            get_name(v): v.data["initial_value"] for v in raw_model.G.variables.values()
        }
    )
intervened_model = intervene(initial_model, interventions)
intervened_samples2 = sample(intervened_model, timepoints=timepoints, num_samples=num_samples)
intervened_samples2

{'beta': tensor([0.0246, 0.0276, 0.0268]),
 'gamma': tensor([0.1510, 0.1344, 0.1397]),
 'Infected_sol': tensor([[  0.9388,   0.8813,   3.4288,  21.9839, 128.9649],
         [  0.9480,   0.8987,   3.5206,  22.5605, 131.9281],
         [  0.9451,   0.8932,   3.4913,  22.3763, 130.9832]]),
 'Recovered_sol': tensor([[0.0731, 0.1418, 0.5882, 1.6580, 7.8154],
         [0.0654, 0.1275, 0.5791, 1.6772, 7.9859],
         [0.0679, 0.1321, 0.5821, 1.6712, 7.9316]]),
 'Susceptible_sol': tensor([[999.9888, 999.9765, 996.9823, 977.3574, 864.2188],
         [999.9866, 999.9738, 996.9002, 976.7620, 861.0856],
         [999.9890, 999.9745, 996.9268, 976.9521, 862.0846]])}

In [6]:
sampled_params = {'beta_param':tensor( [0.2, 0.25, 0.15]), 'gamma_param': tensor([0.3, 0.25, 0.35])}

In [7]:



expected_intervals = {
    'beta_param': [
        {'start': -np.inf, 'end': 1.1, 'param_values': [0.2, 0.25, 0.15]},
        {'start': 1.1, 'end': 1.3, 'param_values': [1.0, 1.0, 1.0]},
        {'start': 1.3, 'end': np.inf, 'param_values': [2.0, 2.0, 2.0]}],
    'gamma_param': [
        {'start': -np.inf, 'end': 1.4, 'param_values': [0.3, 0.25, 0.35]},
        {'start': 1.4, 'end': 2.1, 'param_values': [0.3, 0.3, 0.3]},
        {'start': 2.1, 'end': np.inf, 'param_values': [0.1, 0.1, 0.1]}]}

#assert expected_intervals == 
interventions_and_sampled_params_to_interval(interventions, sampled_params)

{'beta_param': [{'start': -inf,
   'end': 1.1,
   'param_values': tensor([0.2000, 0.2500, 0.1500])},
  {'start': 1.1, 'end': 1.3, 'param_values': [1.0, 1.0, 1.0]},
  {'start': 1.3, 'end': inf, 'param_values': [2.0, 2.0, 2.0]}],
 'gamma_param': [{'start': -inf,
   'end': 1.4,
   'param_values': tensor([0.3000, 0.2500, 0.3500])},
  {'start': 1.4, 'end': 2.1, 'param_values': [0.3, 0.3, 0.3]},
  {'start': 2.1, 'end': inf, 'param_values': [0.1, 0.1, 0.1]}]}

In [9]:
tensor([1., 2., 3.]).

[tensor(1.), tensor(2.), tensor(3.)]

Given a set of values at particular intervals,  a set of initial samples and a set of timepoints, generate a list of timepoints where the parameter has the same value at each timepoint in the interval.


import bisect

def assign_interventions_to_timepoints(interventions: dict, timepoints: Iterable[float], initial_values: dict) -> dict:
    # transform samples into intervals and parameter values
    param_interval_dict = intervention_to_interval(interventions, initial_values)
    result = {}
    for param, interval_dict in param_interval_dict.items():
        intervals = [(d['start'], d['end']) for d in interval_dict]
        values_sets = [d['expected_value'] for  d in interval_dict]

        # generate list of parameter values at each timepoint
        result[param] = []
        for values in zip(*values_sets):
            for t in timepoints:
                i = bisect.bisect(intervals, (t,)) - 1
                if 0 <= i < len(intervals) and intervals[i][0] <= t < intervals[i][1]:
                    result[param].append(values[i])
                else:
                    result[param].append( None)
    return result


In [10]:
for param, interval_dict in expected_intervals.items():
    intervals = [(d['start'], d['end']) for d in interval_dict]
    param_values = [d['param_values'] for d in interval_dict]
intervals

[(-inf, 1.4), (1.4, 2.1), (2.1, inf)]

In [14]:
param_values

[[0.3, 0.25, 0.35], [0.3, 0.3, 0.3], [0.1, 0.1, 0.1]]

In [19]:
import bisect
result = {}
for param, interval_dict in expected_intervals.items():
    intervals = [(d['start'], d['end']) for d in interval_dict]
    param_values = [d['param_values'] for d in interval_dict]

    # generate list of parameter values at each timepoint
    result[param] = []    
    for values in zip(*param_values):
        for t in timepoints:
            # find the interval that contains the timepoint
            i = bisect.bisect(intervals, (t,)) - 1
            if 0 <= i < len(intervals) and intervals[i][0] <= t < intervals[i][1]:
                result[param].append(values[i])
            else:
                # If the timepoint is not in an interval, assign None. This should never happen
                result[param].append(None)
result

{'beta_param': [0.2,
  0.2,
  2.0,
  2.0,
  2.0,
  0.25,
  0.25,
  2.0,
  2.0,
  2.0,
  0.15,
  0.15,
  2.0,
  2.0,
  2.0],
 'gamma_param': [0.3,
  0.3,
  0.3,
  0.1,
  0.1,
  0.25,
  0.25,
  0.3,
  0.1,
  0.1,
  0.35,
  0.35,
  0.3,
  0.1,
  0.1]}

In [20]:
interventions, timepoints, initial_values

([(1.1, 'beta', 1.0),
  (2.1, 'gamma', 0.1),
  (1.3, 'beta', 2.0),
  (1.4, 'gamma', 0.3)],
 [0.5, 1.0, 2.0, 3.0, 4.0],
 {'beta_param': [0.2, 0.25, 0.15], 'gamma_param': [0.3, 0.25, 0.35]})

In [17]:
display(intervened_samples2)
parameters = {}
for name, sample in intervened_samples2.items():
    if sample.ndim == 1:
        parameters[f"{name}_param"] = (
                sample.data.detach().cpu().numpy().astype(np.float64)
            )
parameters

{'beta': tensor([0.0263, 0.0271, 0.0246]),
 'gamma': tensor([0.1308, 0.1342, 0.1359]),
 'Infected_sol': tensor([[  0.9491,   0.9008,   3.5336,  22.6421, 132.3466],
         [  0.9478,   0.8984,   3.5195,  22.5536, 131.8928],
         [  0.9459,   0.8946,   3.5016,  22.4412, 131.3172]]),
 'Recovered_sol': tensor([[0.0637, 0.1242, 0.5759, 1.6780, 8.0082],
         [0.0653, 0.1273, 0.5787, 1.6764, 7.9834],
         [0.0661, 0.1286, 0.5784, 1.6706, 7.9482]]),
 'Susceptible_sol': tensor([[999.9871, 999.9746, 996.8905, 976.6799, 860.6453],
         [999.9868, 999.9745, 996.9033, 976.7705, 861.1242],
         [999.9880, 999.9767, 996.9209, 976.8878, 861.7342]])}

{'beta_param': array([0.02633861, 0.02711203, 0.02463129]),
 'gamma_param': array([0.13076352, 0.13422994, 0.13593858])}

In [16]:
expected_intervened_values = {
    'beta_param': [0.2, 0.2, 2.0, 2.0, 2.0, 0.25, 0.25, 2.0, 2.0, 2.0, 0.15, 0.15, 2.0, 2.0, 2.0],
    'gamma_param': [0.3, 0.3, 0.3, 0.1, 0.1, 0.25, 0.25, 0.3, 0.1, 0.1, 0.35, 0.35, 0.3, 0.1, 0.1]
}
assert expected_intervened_values == assign_interventions_to_timepoints(interventions, timepoints, sampled_params)

In [20]:
from pyciemss.utils.interface_utils import convert_to_output_format
intervened_samples = {
            'beta': tensor([0.0263, 0.0271, 0.0246]),
            'gamma': tensor([0.1308, 0.1342, 0.1359]),
            'Infected_sol': tensor([[  0.9491,   0.9008,   3.5336,  22.6421, 132.3466],
                                    [  0.9478,   0.8984,   3.5195,  22.5536, 131.8928],
                                    [  0.9459,   0.8946,   3.5016,  22.4412, 131.3172]]),
            'Recovered_sol': tensor([[0.0637, 0.1242, 0.5759, 1.6780, 8.0082],
                                     [0.0653, 0.1273, 0.5787, 1.6764, 7.9834],
                                     [0.0661, 0.1286, 0.5784, 1.6706, 7.9482]]),
            'Susceptible_sol': tensor([[999.9871, 999.9746, 996.8905, 976.6799, 860.6453],
                                       [999.9868, 999.9745, 996.9033, 976.7705, 861.1242],
                                       [999.9880, 999.9767, 996.9209, 976.8878, 861.7342]])}
df = convert_to_output_format(intervened_samples, timepoints, interventions)
df2 = pd.read_csv('test/test_utils/expected_output_format.csv') 
assert df2.equals(convert_to_output_format(intervened_samples, timepoints, interventions))

import pandas as pd

AssertionError: 

In [36]:

self_initial_time = 0.0
initial_state = {
            "Susceptible": 0.99,
            "Infected": 0.01,
            "Recovered": 0.0,
}
timepoints = [1.0, 1.1, 1.2, 1.3]

interventions=[(1e-6, "beta", 1.0), (2e-6, "gamma", 0.1)]
ASKENET_PATH = "https://raw.githubusercontent.com/DARPA-ASKEM/Model-Representations/main/petrinet/examples/sir_typed.json"

intervened_samples = load_and_sample_petri_model(ASKENET_PATH, num_samples, timepoints, interventions=self_interventions, start_state=initial_state)
intervened_samples.to_csv('test/test_petrinet_ode/expected_intervened_samples.csv', index=False)
intervened_samples

,timepoint_id,sample_id,beta_param,gamma_param,Infected_sol,Recovered_sol,Susceptible_sol
0,0,0,1.0,0.1,0.024187,0.001608,0.974205
1,1,0,1.0,0.1,0.026393,0.001861,0.971746
2,2,0,1.0,0.1,0.028794,0.002137,0.969069
3,3,0,1.0,0.1,0.031403,0.002438,0.966159
4,0,1,1.0,0.1,0.024187,0.001608,0.974205
5,1,1,1.0,0.1,0.026393,0.001861,0.971746
6,2,1,1.0,0.1,0.028794,0.002137,0.969069
7,3,1,1.0,0.1,0.031403,0.002438,0.966159
8,0,2,1.0,0.1,0.024187,0.001608,0.974205
9,1,2,1.0,0.1,0.026393,0.001861,0.971746


In [21]:
df2

,timepoint_id,sample_id,beta_param,gamma_param,Infected_sol,Recovered_sol,Susceptible_sol
0,0,0,0.0263,0.1308,0.949100,0.0637,999.987122
1,1,0,0.0263,0.1308,0.900800,0.1242,999.974609
2,2,0,2.0000,0.3000,3.533600,0.5759,996.890503
3,3,0,2.0000,0.1000,22.642099,1.6780,976.679871
4,4,0,2.0000,0.1000,132.346603,8.0082,860.645325
5,0,1,0.0271,0.1342,0.947800,0.0653,999.986816
6,1,1,0.0271,0.1342,0.898400,0.1273,999.974487
7,2,1,2.0000,0.3000,3.519500,0.5787,996.903320
8,3,1,2.0000,0.1000,22.553600,1.6764,976.770508
9,4,1,2.0000,0.1000,131.892807,7.9834,861.124207


In [26]:
from pandas.testing import assert_frame_equal

assert_frame_equal(df, df2, check_exact=False, atol=1e-5, rtol=1e-5)


## load_and_calibrate_and_sample_petri_model

In [7]:
data_path = os.path.join(DEMO_PATH, "data.csv")
num_samples = 100
timepoints = [0.0, 1.0, 2.0, 3.0, 4.0]

# Run the calibration and sampling
calibrated_samples = load_and_calibrate_and_sample_petri_model(
    ASKENET_PATH,
    data_path,
    num_samples,
    timepoints=timepoints,
    verbose=True,
)

# Save results
calibrated_samples.to_csv(
    os.path.join(DEMO_PATH, "results_petri/calibrated_sample_results.csv"), index=False
)

iteration 0: loss = 37.94811773300171
iteration 25: loss = 34.9170196056366
iteration 50: loss = 32.69080424308777
iteration 75: loss = 32.41730046272278
iteration 100: loss = 32.44331431388855
iteration 125: loss = 32.692076683044434
iteration 150: loss = 32.61838483810425
iteration 175: loss = 32.56607246398926
iteration 200: loss = 33.014737367630005
iteration 225: loss = 32.58268332481384
iteration 250: loss = 32.89216685295105
iteration 275: loss = 31.961626768112183
iteration 300: loss = 32.6565682888031
